## ImagePoemMatch

- input: 이미지키워드 토크나이징 데이터 (json) + 시 토크나이징 데이터 (json)
- output: 이미지 키워드와 시 키워드의 코사인 유사도가 가장 높은 이미지와 시 매칭 (json)

### 1. 필요한 라이브러리 import

In [20]:
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import json
import fasttext
from numpy import dot
from numpy.linalg import norm

### 2. 필요한 데이터 load

In [2]:
# 이미지 토픽 모음
image_topic = ['beach', 'cave', 'island', 'lake', 'mountain', 'amusement park', 'palace', 'park', 'restaurant', 'tower']

In [3]:
# 이미지 키워드 토크나이징 데이터 저장 리스트
image_tokens = []

In [ ]:
# 이미지 키워드 토크나이징 데이터 로드
for topic in image_topic:
    with open('../data/{topic}_token.json'.format(topic=topic), 'rb') as f:
        image_tokens.append(json.load(f))

In [7]:
# 시 키워드 토크나이징 데이터 로드
with open('../data/poem_token.pkl', 'rb') as f:
    poem_token = pickle.load(f)

### 2-1. 테스트 코드

In [9]:
# 테스트 코드
with open('../data/beach_token.json', 'rb') as f:
    beach_keyword_token = json.load(f)

In [25]:
# 데이터 모양 확인
beach_keyword_token[0]

{'label': 'beach',
 'img_name': '1',
 'keyword': [['바닷가/NNG_'],
  ['모래/NNG_'],
  ['바다/NNG_'],
  ['물/NNG_', '이/JKS_', '몸/NNG_'],
  ['여름/NNG_'],
  ['대양/NNG_'],
  ['해/NNG_'],
  ['재미/NNG_'],
  ['분명히/MAG_'],
  ['레저/NNG_', '(/SS_', '시간/NNG_', '꺼/VV_', '짐/NNG_', ')/SS_'],
  ['여행/NNG_'],
  ['휴가/NNG_'],
  ['사랑/NNG_'],
  ['아이/NNG_'],
  ['밀리/VV_', '어/EC_', '오/VX_', '는/ETM_', '파도/NNG_'],
  ['자연/NNG_'],
  ['연안/NNG_'],
  ['천국/NNG_'],
  ['자유/NNG_', '(/SS_', '상태/NNP_', ')/SS_'],
  ['남성/NNG_'],
  ['ᄉ/NNG_', 'ᅩᆨ초해수ᄋ/SH_', 'ᅭᆨ자/NNP_', 'ᆼ/SH_'],
  ['ᄀ/NNG_', 'ᅡ족/SH_', '여해/NNP_', 'ᆼ/SH_'],
  ['ᄀ/NNG_', 'ᅧ울/SH_', '바ᄃ/NNP_', 'ᅡ/SH_'],
  ['ᄉ/NNG_', 'ᅩᆨ초해수ᄋ/SH_', 'ᅭᆨ자/NNP_', 'ᆼ/SH_'],
  ['ᄀ/NNG_', 'ᅡ족/SH_', '여해/NNP_', 'ᆼ/SH_'],
  ['ᄀ/NNG_', 'ᅧ울/SH_', '바ᄃ/NNP_', 'ᅡ/SH_'],
  ['ᄉ/NNG_', 'ᅩᆨ초해수ᄋ/SH_', 'ᅭᆨ자/NNP_', 'ᆼ/SH_'],
  ['ᄀ/NNG_', 'ᅡ족/SH_', '여해/NNP_', 'ᆼ/SH_'],
  ['ᄀ/NNG_', 'ᅧ울/SH_', '바ᄃ/NNP_', 'ᅡ/SH_'],
  ['ᄇ/NNG_',
   'ᅡ/SH_',
   'ᄃ/SL_',
   'ᅡᄂ/SH_',
   'ᅳ/NNP_',
   'ᆫ/SH_'

### 3. 데이터에서 명사인 키워드만 추출

In [26]:
def del_none(x):
    d = []
    for j in tqdm(x):
        c = []
        for i in j:
            if i == None:
                pass
            else:
                c.append(i)
        d.append(c)
    return d

In [27]:
def select_noun_va(x):
    new_a = []
    for j in tqdm(x):
        aa = []
        for i in j:   
            if 'NNG' in i:
                aa.append(i)
        new_a.append(aa)
    return new_a

### 3-1. 이미지

In [ ]:
def get_noun_keyword(img_data):
    key_tok = []
    for i in range(len(img_data)):
        key_tok.append(img_data[i]['keyword'])
    flatten_key_tok = []
    for i in key_tok:
        flatten_key_tok.append([y for x in i for y in x])
    noun_kwd = select_noun_va(del_none(flatten_key_tok))
    for i in tqdm(range(len(img_data))):
        img_data[i]['keyword'] = 0
        img_data[i]['keyword'] = list(set(noun_kwd[i]))
    return img_data #noun_keyword로 호출

### 3-2. 시

명사 토큰이 없을 때 해당 시를 버리겠다는 의미로 'ㄱ'을 넣어준다.

In [ ]:
# 도연
def get_noun_poem(poem_data):
    flatten_poem_tok = []
    for i in poem_data:
        flatten_poem_tok.append([y for x in i for y in x])
    noun_poem = select_noun_va(del_none(flatten_poem_tok))
    # 시의 경우, 명사 토큰이 없는 경우, 임의로 'ㄱ'을 넣어준다. 해당 시를 버리겠다는 것
    for i in range(len(noun_poem)):
        if len(noun_poem[i]) == 0:
            noun_poem[i] = ['ㄱ']       
    return poem_data 

### 4. fasttext 모델로 word vector 생성

In [28]:
model = fasttext.load_model('../model/wiki.ko/wiki.ko.bin')

In [29]:
model.get_dimension()

300

### 4-1. 이미지

### 4-2. 시

### 5. 코사인 유사도 구하기

In [ ]:
def cosine_similarity(x, y):
    return np.dot(x, y) / (np.sqrt(np.dot(x, x)) * np.sqrt(np.dot(y, y)))

In [ ]:
def return_index(vector_keyword, vector_poem):
    scores = []
    for i in range(len(vector_poem)):
        score = cosine_similarity(vector_keyword, vector_poem[i])
        scores.append(score)
    index = scores.index(max(scores))
    return index

In [ ]:
# --------------------------------------------------------------------------------------------------- 수정 필요

In [ ]:
# 각 단어의 300차원으로 임베딩된 데이터
total = []
for i in tqdm(range(len(noun_poem))):
    sen = []
    total.append(sen)
    for j in range(len(noun_poem[i])):
        #if noun_poem[i][j] != word:
            sen.append(model.get_word_vector(noun_poem[i][j]))

In [ ]:
# 시 단위로 임베딩 벡터들을 더해준다. 그것을 시에 대한 벡터값으로 정의

In [ ]:
new_total = np.array([np.array(i) for i in total ])

In [ ]:
new_total_samplesum = [ np.sum(i, axis=0) for i in new_total]

In [ ]:
new_total_samplesum = np.array(new_total_samplesum)

In [ ]:
vector_poem = new_total_samplesum

In [ ]:
vector_poem[1].shape

### beach

In [ ]:
# 이미지 키워드의 명사 토큰 데이터
with open('final_beach.json', 'rb') as f:
     json_data = json.load(f)

In [ ]:
# 각 키워드에 대해서 300차원으로 임베딩된 데이터

In [ ]:
tot = []
for i in tqdm(range(len(json_data))):
    sen = []
    tot.append(sen)
    for j in range(len(json_data[i]['keyword'])):
        #if json_data[i]['keyword'][j] != word:
            sen.append(model.get_word_vector(json_data[i]['keyword'][j]))

In [ ]:
# 이것 또한 마찬가지로 이미지(keyword) 를 대표하는 하나의 벡터로 나타내준다.

In [ ]:
new_tot = np.array([np.array(i) for i in tot ])

In [ ]:
new_tot_samplesum = [ np.sum(i, axis=0) for i in new_tot]

In [ ]:
new_tot_samplesum = np.array(new_tot_samplesum)

In [ ]:
vector_keyword = new_tot_samplesum

In [ ]:
vector_keyword[1].shape

#### cosin sim

In [ ]:
# i : keyword_Vector로 이미지에 맞는, 가장 유사도가 높은 poem_vector를 구하고 그 시의 인덱스를 append

In [ ]:
index_list = []
for i in range(len(vector_keyword)):
    index = return_index(vector_keyword[i], vector_poem)
    index_list.append(index)

In [ ]:
len(index_list)

In [ ]:
len(set(index_list))  # 매칭된 시 개수

## for 태욱 최종 json 저장

In [ ]:
# 해당 match되는 시를 match_poem에 저장
match_poem = []
for i in index_list:
    match_poem.append(poem_token[i]) 

In [ ]:
with open('final_beach_token.json', 'rb') as f:
     final_beach_token = json.load(f)

In [ ]:
# json_data에 넣어준다.
for i in tqdm(range(len(match_poem))):
    final_beach_token[i]['text'] = 0
    final_beach_token[i]['text'] = match_poem[i]

In [ ]:
for i in range(len(final_beach_token)):
    final_beach_token[i]['image'] = final_beach_token[i]['img_name']

In [ ]:
for i in range(len(final_beach_token)):
    del final_beach_token[i]['img_name']

In [ ]:
final_beach_token[0]

In [ ]:
# 이건lake에 poem이 이미 포함되어 있어서 지우는 코드
#for i in range(len(final_lake_token)):
#    del final_lake_token[i]['poem']

In [ ]:
with open('real_final_beach_token.json', 'w', encoding='utf-8') as f:
    json.dump(final_beach_token, f)

### match poem list 만들기

In [ ]:
index_list = []
for i in range(len(vector_keyword)):
    index = return_index(vector_keyword[i], vector_poem)
    index_list.append(index)

In [ ]:
len(index_list)

In [ ]:
len(set(index_list))  # 매칭된 시 개수

## for 태욱 최종 JSON 저장

In [ ]:
# 해당 match되는 시를 match_poem에 저장
match_poem = []
for i in index_list:
    match_poem.append(poem_token[i]) 

In [ ]:
with open('final_amusement_token.json', 'rb') as f:
     final_amusement_token = json.load(f)

In [ ]:
# json_data에 넣어준다.
for i in tqdm(range(len(match_poem))):
    final_amusement_token[i]['text'] = 0
    final_amusement_token[i]['text'] = match_poem[i]  

In [ ]:
final_amusement_token[0]

In [ ]:
with open('real_final_amusement_token.json', 'w', encoding='utf-8') as f:
    json.dump(final_amusement_token, f)